In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import numpy as np
import pandas as pd
import pandas_ta as ta

from gplearn.extra_functions_2 import *

In [28]:
data = np.random.rand(4000, 300) 
# data[::10] = np.nan
data

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.71678945, 0.34191978, 0.66166719, ..., 0.86792586, 0.28516771,
        0.91258029],
       [0.97680916, 0.46824338, 0.1693206 , ..., 0.70472278, 0.88244676,
        0.04373355],
       ...,
       [0.07083991, 0.58968004, 0.99557554, ..., 0.43394225, 0.558227  ,
        0.51898742],
       [0.45355648, 0.89691758, 0.4405944 , ..., 0.09590089, 0.52226284,
        0.51846504],
       [0.53430711, 0.62214206, 0.29453916, ..., 0.46957347, 0.11628285,
        0.51454718]])

In [61]:
%timeit ta_APO(data, 12, 26)

4.41 ms ± 59.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%timeit pd.DataFrame(data).apply(lambda x: ta.apo(x, 12, 26)).to_numpy(), ta_APO(data, 12, 26)

19.9 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
tszs_add(data, data, 10)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.414214,1.414214,-1.414214,1.414214,-1.414214,-1.414214,-1.414214,-1.414214,1.414214,-1.414214,...,-1.414214,-1.414214,-1.414214,1.414214,-1.414214,-1.414214,1.414214,-1.414214,1.414214,-1.414214
3,-2.223199,2.158347,1.582851,2.047867,2.161183,-0.522867,1.312992,-1.692102,-0.362275,2.291570,...,-1.751492,1.200351,1.114515,-1.542450,2.219834,-0.229378,1.428300,-2.259304,1.321884,0.338389
4,1.251764,-1.741461,0.710261,2.008582,-2.040553,2.018002,1.109202,-1.488596,2.616949,1.931962,...,-2.976375,1.468792,1.325200,2.952457,2.801184,-0.740369,1.897936,1.020592,-0.979391,-1.298914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.977069,-2.858512,0.627473,3.018700,-0.037654,-4.043600,4.125186,-2.063191,2.477306,-0.181938,...,2.485139,2.455926,0.634421,-0.942870,0.770411,-2.388047,1.208820,0.218349,-2.398125,-2.845598
3996,3.459116,-2.096784,0.558922,1.357510,0.958124,1.146716,3.238637,-1.133322,-1.570725,2.171786,...,-2.643669,-1.630887,1.311442,-0.026567,0.285504,0.655375,-3.166899,3.517956,-1.260149,0.925890
3997,-1.390883,0.475775,2.281487,1.042276,1.405517,-1.022747,1.449255,1.971836,1.749409,0.409656,...,0.610464,2.560196,-1.945072,2.249340,-1.447156,1.793049,0.534216,-0.419657,0.603322,-0.627360
3998,1.053457,2.134156,-0.562199,1.409421,0.930502,-0.968787,-2.038775,-1.911253,-1.776386,-2.120505,...,-2.915667,-2.350354,1.727287,1.879011,-2.028822,2.769861,0.701740,-2.282158,0.793274,-0.365767


In [38]:
%timeit tszs_add(data, data, 10)

64.2 ms ± 788 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
pd.DataFrame(ta_MACDs(data, 12, 26, 9))

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
data

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.71678945, 0.34191978, 0.66166719, ..., 0.86792586, 0.28516771,
        0.91258029],
       [0.97680916, 0.46824338, 0.1693206 , ..., 0.70472278, 0.88244676,
        0.04373355],
       ...,
       [0.07083991, 0.58968004, 0.99557554, ..., 0.43394225, 0.558227  ,
        0.51898742],
       [0.45355648, 0.89691758, 0.4405944 , ..., 0.09590089, 0.52226284,
        0.51846504],
       [0.53430711, 0.62214206, 0.29453916, ..., 0.46957347, 0.11628285,
        0.51454718]])

In [55]:
talib.MACD(data[:, 1], 12, 26, 9)[0]

array([nan, nan, nan, ..., nan, nan, nan])

In [56]:
data[:, 1]

array([       nan, 0.34191978, 0.46824338, ..., 0.58968004, 0.89691758,
       0.62214206])